In [2]:
!pip install timm torch==1.7.0 torchvision==0.8.1
!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
!python pytorch-xla-env-setup.py


     |████████████████████████████████| 548 kB 2.2 MB/s eta 0:00:01
     |████████████████████████████████| 776.7 MB 2.5 kB/s s eta 0:00:01.1 MB/s eta 0:00:01
     |████████████████████████████████| 12.7 MB 41.8 MB/s eta 0:00:01
  Attempting uninstall: torch
    Found existing installation: torch 1.7.1+cpu
    Uninstalling torch-1.7.1+cpu:
      Successfully uninstalled torch-1.7.1+cpu
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.8.2+cpu
    Uninstalling torchvision-0.8.2+cpu:
      Successfully uninstalled torchvision-0.8.2+cpu
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.8.1 requires torch==1.7.1, but you have torch 1.7.0 which is incompatible.
torchaudio 0.7.2 requires torch==1.7.1, but you have torch 1.7.0 which is incompatible.
curl: /opt/conda/lib/libcurl.so.4: no version information available (required

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("ggplot")

import torch
import torch.nn as nn
import torchvision.transforms as transforms

import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.distributed.xla_multiprocessing as xmp
import torch_xla.distributed.parallel_loader as pl

import timm

import gc
import os
import time
import random
from datetime import datetime

from PIL import Image
from tqdm.notebook import tqdm
from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split

[('__call__', <function LevelMapper.__call__ at 0x7fe8565b1170>), ('__init__', <function LevelMapper.__init__ at 0x7fe8565b10e0>)]
[('__call__', <function BalancedPositiveNegativeSampler.__call__ at 0x7fe88165ba70>), ('__init__', <function BalancedPositiveNegativeSampler.__init__ at 0x7fe88165b9e0>)]
[('__init__', <function BoxCoder.__init__ at 0x7fe88166b290>), ('decode', <function BoxCoder.decode at 0x7fe88166b440>), ('decode_single', <function BoxCoder.decode_single at 0x7fe88166b4d0>), ('encode', <function BoxCoder.encode at 0x7fe88166b320>), ('encode_single', <function BoxCoder.encode_single at 0x7fe88166b3b0>)]
[('__call__', <function Matcher.__call__ at 0x7fe88166b5f0>), ('__init__', <function Matcher.__init__ at 0x7fe881660f80>), ('set_low_quality_matches_', <function Matcher.set_low_quality_matches_ at 0x7fe88166b170>)]
[('__init__', <function ImageList.__init__ at 0x7fe88166b0e0>), ('to', <function ImageList.to at 0x7fe88166b710>)]
[('__init__', <function Timebase.__init__ at

In [4]:
# For parallelization in TPUs
os.environ["XLA_USE_BF16"] = "1"
os.environ["XLA_TENSOR_ALLOCATOR_MAXSIZE"] = "100000000"

In [5]:
def seed_everything(seed):
    """
    Seeds basic parameters for reproductibility of results
    
    Arguments:
        seed {int} -- Number of the seed
    """
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(1001)

In [6]:
#make train, test sets
from sklearn import preprocessing

DATA_PATH = "../input/sdss-images/data.csv"
df = pd.read_csv(DATA_PATH)

le = preprocessing.LabelEncoder()
df['class'] = le.fit_transform(df['class'])

train,val = train_test_split(df,test_size=0.2,random_state=2022)
train = train.reset_index().drop(['index'],axis=1)
val = val.reset_index().drop(['index'],axis=1)
val,test= train_test_split(val,test_size=0.5,random_state=2022)
val = val.reset_index().drop(['index'],axis=1)
test = test.reset_index().drop(['index'],axis=1)

In [7]:
le.classes_

array(['GALAXY', 'QSO', 'STAR'], dtype=object)

In [8]:
train.head(2)

,class,image
0,0,image_5516.jpg
1,1,image_1255.jpg


In [9]:
!mkdir images
!mkdir images/val images/train

In [10]:
import shutil

for img in train['image']:
    shutil.copy("../input/sdss-images/images (1)/images/"+img,"./images/train/"+img)
    
for img in val['image']:
    shutil.copy("../input/sdss-images/images (1)/images/"+img,"./images/val/"+img)
    

In [11]:
# general global variables
DATA_PATH = "../input/sdss-images/data.csv"
TRAIN_PATH = "./images/train"
TEST_PATH = "./images/val"
MODEL_PATH = (
    "../input/vit-base-models-pretrained-pytorch/jx_vit_base_p16_224-80ecf9dd.pth"
)

# model specific global variables
IMG_SIZE = 224
BATCH_SIZE = 32 #32
LR = 2e-05
GAMMA = 0.7
N_EPOCHS = 10
wandb_args = {"learning_rate": LR, "epochs": N_EPOCHS, "batch_size": BATCH_SIZE,"gamma":GAMMA,"img_size":IMG_SIZE,"project":"ViT SDSS","name":"Default aug"}

In [12]:
class SDSSDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, mode="train", transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        self.mode = mode
        self.data_dir = "./images/train" if mode == "train" else "./images/val"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        label,img_name = self.df_data[index]
        img_path = os.path.join(self.data_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(img)

        return image, label

In [13]:
# create image augmentations
transforms_train = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomResizedCrop(IMG_SIZE),
        transforms.ToTensor(),
    ]
)

transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ]
)

In [14]:
print("Available Vision Transformer Models: ")
timm.list_models("vit*")

Available Vision Transformer Models: 


['vit_base_patch8_224',
 'vit_base_patch8_224_dino',
 'vit_base_patch8_224_in21k',
 'vit_base_patch16_18x2_224',
 'vit_base_patch16_224',
 'vit_base_patch16_224_dino',
 'vit_base_patch16_224_in21k',
 'vit_base_patch16_224_miil',
 'vit_base_patch16_224_miil_in21k',
 'vit_base_patch16_224_sam',
 'vit_base_patch16_384',
 'vit_base_patch16_plus_240',
 'vit_base_patch16_rpn_224',
 'vit_base_patch32_224',
 'vit_base_patch32_224_clip_laion2b',
 'vit_base_patch32_224_in21k',
 'vit_base_patch32_224_sam',
 'vit_base_patch32_384',
 'vit_base_patch32_plus_256',
 'vit_base_r26_s32_224',
 'vit_base_r50_s16_224',
 'vit_base_r50_s16_224_in21k',
 'vit_base_r50_s16_384',
 'vit_base_resnet26d_224',
 'vit_base_resnet50_224_in21k',
 'vit_base_resnet50_384',
 'vit_base_resnet50d_224',
 'vit_giant_patch14_224',
 'vit_giant_patch14_224_clip_laion2b',
 'vit_gigantic_patch14_224',
 'vit_huge_patch14_224',
 'vit_huge_patch14_224_clip_laion2b',
 'vit_huge_patch14_224_in21k',
 'vit_large_patch14_224',
 'vit_large_

In [15]:
class ViTBase16(nn.Module):
    def __init__(self, n_classes, pretrained=False):

        super(ViTBase16, self).__init__()

        self.model = timm.create_model("vit_base_patch16_224", pretrained=False)
        if pretrained:
            self.model.load_state_dict(torch.load(MODEL_PATH))

        self.model.head = nn.Linear(self.model.head.in_features, n_classes)

    def forward(self, x):
        x = self.model(x)
        return x

    def train_one_epoch(self, train_loader, criterion, optimizer, device):
        # keep track of training loss
        epoch_loss = 0.0
        epoch_accuracy = 0.0

        ###################
        # train the model #
        ###################
        self.model.train()
        for i, (data, target) in enumerate(train_loader):
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
#                 print(target)
                target = torch.tensor(target).to(device, dtype=torch.int64)

            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = self.forward(data)
            # calculate the batch loss
            loss = criterion(output, target)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # Calculate Accuracy
            accuracy = (output.argmax(dim=1) == target).float().mean()
            # update training loss and accuracy
            epoch_loss += loss
            epoch_accuracy += accuracy

            # perform a single optimization step (parameter update)
            if device.type == "xla":
                xm.optimizer_step(optimizer)

                if i % 20 == 0:
                    xm.master_print(f"\tBATCH {i+1}/{len(train_loader)} - LOSS: {loss}")

            else:
                optimizer.step()

        return epoch_loss / len(train_loader), epoch_accuracy / len(train_loader)

    def validate_one_epoch(self, valid_loader, criterion, device):
        # keep track of validation loss
        valid_loss = 0.0
        valid_accuracy = 0.0

        ######################
        # validate the model #
        ######################
        self.model.eval()
        for data, target in valid_loader:
            # move tensors to GPU if CUDA is available
            if device.type == "cuda":
                data, target = data.cuda(), target.cuda()
            elif device.type == "xla":
                data = data.to(device, dtype=torch.float32)
                target = torch.tensor(target).to(device, dtype=torch.int64)

            with torch.no_grad():
                # forward pass: compute predicted outputs by passing inputs to the model
                output = self.model(data)
                # calculate the batch loss
                loss = criterion(output, target)
                # Calculate Accuracy
                accuracy = (output.argmax(dim=1) == target).float().mean()
                # update average validation loss and accuracy
                valid_loss += loss
                valid_accuracy += accuracy

        return valid_loss / len(valid_loader), valid_accuracy / len(valid_loader)

In [16]:
import wandb
import os
from kaggle_secrets import UserSecretsClient
model = ViTBase16(n_classes=3, pretrained=True)

def fit_tpu(
    model, epochs, device, criterion, optimizer, train_loader, valid_loader=None
):

    valid_loss_min = np.Inf  # track change in validation loss

    # keeping track of losses as it happen
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []
    
    


    for epoch in range(1, epochs + 1):
        gc.collect()
        para_train_loader = pl.ParallelLoader(train_loader, [device])
        
        xm.master_print(f"{'='*50}")
        xm.master_print(f"EPOCH {epoch} - TRAINING...")
        train_loss, train_acc = model.train_one_epoch(
            para_train_loader.per_device_loader(device), criterion, optimizer, device
        )
        xm.master_print(
            f"\n\t[TRAIN] EPOCH {epoch} - LOSS: {train_loss}, ACCURACY: {train_acc}\n"
        )
        train_losses.append(train_loss)
        train_accs.append(train_acc)
        gc.collect()
        
        if valid_loader is not None:
            gc.collect()
            para_valid_loader = pl.ParallelLoader(valid_loader, [device])
            xm.master_print(f"EPOCH {epoch} - VALIDATING...")
            valid_loss, valid_acc = model.validate_one_epoch(
                para_valid_loader.per_device_loader(device), criterion, device
            )
            xm.master_print(f"\t[VALID] LOSS: {valid_loss}, ACCURACY: {valid_acc}\n")
            valid_losses.append(valid_loss)
            valid_accs.append(valid_acc)
            gc.collect()

            # save model if validation loss has decreased
            if valid_loss <= valid_loss_min and epoch != 1:
                xm.master_print(
                    "Validation loss decreased ({:.4f} --> {:.4f}).  Saving model ...".format(
                        valid_loss_min, valid_loss
                    )
                )

            valid_loss_min = valid_loss
    logs={
        "train_loss": train_losses,
        "valid_losses": valid_losses,
        "train_acc": train_accs,
        "valid_acc": valid_accs,
    }

    return logs

In [17]:
def _run():
    train_dataset = SDSSDataset(train, transforms=transforms_train)
    valid_dataset = SDSSDataset(val, transforms=transforms_valid,mode="valid",)

    train_sampler = torch.utils.data.distributed.DistributedSampler(
        train_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=True,
    )

    valid_sampler = torch.utils.data.distributed.DistributedSampler(
        valid_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False,
    )

    train_loader = torch.utils.data.DataLoader(
        dataset=train_dataset,
        batch_size=BATCH_SIZE,
        sampler=train_sampler,
        drop_last=True,
        num_workers=8,
    )

    valid_loader = torch.utils.data.DataLoader(
        dataset=valid_dataset,
        batch_size=BATCH_SIZE,
        sampler=valid_sampler,
        drop_last=True,
        num_workers=8,
    )

    criterion = nn.CrossEntropyLoss()
    device = xm.xla_device()
    model.to(device)
    
    lr = LR * xm.xrt_world_size()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    xm.master_print(f"INITIALIZING TRAINING ON {xm.xrt_world_size()} TPU CORES")
    start_time = datetime.now()
    xm.master_print(f"Start Time: {start_time}")

    logs = fit_tpu(
        model=model,
        epochs=N_EPOCHS,
        device=device,
        criterion=criterion,
        optimizer=optimizer,
        train_loader=train_loader,
        valid_loader=valid_loader,
    )

    xm.master_print(f"Execution time: {datetime.now() - start_time}")

    xm.master_print("Saving Model")
    xm.save(
        model.state_dict(), f'model_5e_{datetime.now().strftime("%Y%m%d-%H%M")}.pth'
    )

In [18]:
%%time

import warnings
warnings.filterwarnings("ignore")

# Start training processes
def _mp_fn(rank, flags):
    torch.set_default_tensor_type("torch.FloatTensor")
    a = _run()


# _run()
FLAGS = {}
xmp.spawn(_mp_fn, args=(FLAGS,), nprocs=8, start_method="fork")

2022-10-24 15:35:06.842640: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.849946: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.870753: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.886190: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.903731: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.919535: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739
2022-10-24 15:35:06.937509: I tensorflow

INITIALIZING TRAINING ON 8 TPU CORES
Start Time: 2022-10-24 15:35:17.511961
EPOCH 1 - TRAINING...


2022-10-24 15:35:21.325088: I tensorflow/compiler/xla/xla_client/computation_client.cc:195] Fetching mesh configuration for worker tpu_worker:0 from mesh service at 304b85e62c1a:54739
2022-10-24 15:35:21.337835: I torch_xla/csrc/tensor_util.cpp:28] Using BF16 data type for floating point values
2022-10-24 15:35:21.706392: I tensorflow/compiler/xla/xla_client/computation_client.cc:195] Fetching mesh configuration for worker tpu_worker:0 from mesh service at 304b85e62c1a:54739
2022-10-24 15:35:21.725354: I torch_xla/csrc/tensor_util.cpp:28] Using BF16 data type for floating point values
2022-10-24 15:35:22.522835: I tensorflow/compiler/xla/xla_client/computation_client.cc:195] Fetching mesh configuration for worker tpu_worker:0 from mesh service at 304b85e62c1a:54739
2022-10-24 15:35:22.536342: I torch_xla/csrc/tensor_util.cpp:28] Using BF16 data type for floating point values
2022-10-24 15:35:22.765674: I tensorflow/compiler/xla/xla_client/computation_client.cc:195] Fetching mesh config

	BATCH 1/31 - LOSS: 1.2734375
	BATCH 21/31 - LOSS: 0.212890625

	[TRAIN] EPOCH 1 - LOSS: 0.44140625, ACCURACY: 0.80859375

EPOCH 1 - VALIDATING...
	[VALID] LOSS: 0.1689453125, ACCURACY: 0.95703125

EPOCH 2 - TRAINING...
	BATCH 1/31 - LOSS: 0.150390625
	BATCH 21/31 - LOSS: 0.375

	[TRAIN] EPOCH 2 - LOSS: 0.251953125, ACCURACY: 0.9140625

EPOCH 2 - VALIDATING...
	[VALID] LOSS: 0.11962890625, ACCURACY: 0.96875

Validation loss decreased (0.1689 --> 0.1196).  Saving model ...
EPOCH 3 - TRAINING...
	BATCH 1/31 - LOSS: 0.208984375
	BATCH 21/31 - LOSS: 0.296875

	[TRAIN] EPOCH 3 - LOSS: 0.2236328125, ACCURACY: 0.91015625

EPOCH 3 - VALIDATING...
	[VALID] LOSS: 0.08984375, ACCURACY: 0.96875

Validation loss decreased (0.1196 --> 0.0898).  Saving model ...
EPOCH 4 - TRAINING...
	BATCH 1/31 - LOSS: 0.08447265625
	BATCH 21/31 - LOSS: 0.2119140625

	[TRAIN] EPOCH 4 - LOSS: 0.2109375, ACCURACY: 0.9296875

EPOCH 4 - VALIDATING...
	[VALID] LOSS: 0.10302734375, ACCURACY: 0.95703125

EPOCH 5 - TRAINING

2022-10-24 15:42:22.672683: I tensorflow/compiler/xla/xla_client/mesh_service.cc:234] Waiting to connect to client mesh master (300 seconds) 304b85e62c1a:54739


CPU times: user 273 ms, sys: 193 ms, total: 466 ms
Wall time: 7min 17s


In [19]:
!ls

__notebook_source__.ipynb
images
model_5e_20221024-1542.pth
pytorch-xla-env-setup.py
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl


In [20]:
!rm -r images

# Inference

In [21]:
!ls

__notebook_source__.ipynb
model_5e_20221024-1542.pth
pytorch-xla-env-setup.py
torch-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torch_xla-nightly+20200515-cp37-cp37m-linux_x86_64.whl
torchvision-nightly+20200515-cp37-cp37m-linux_x86_64.whl


In [22]:
model_inference = ViTBase16(n_classes=3, pretrained=True)

In [25]:
model_inference.load_state_dict(torch.load("./model_5e_20221024-1542.pth"))

<All keys matched successfully>

In [54]:
class SDSSDataset(torch.utils.data.Dataset):
    """
    Helper Class to create the pytorch dataset
    """

    def __init__(self, df, data_path=DATA_PATH, transforms=None):
        super().__init__()
        self.df_data = df.values
        self.data_path = data_path
        self.transforms = transforms
        
        self.data_dir = "./images/test"

    def __len__(self):
        return len(self.df_data)

    def __getitem__(self, index):
        label,img_name = self.df_data[index]
        img_path = os.path.join(self.data_dir, img_name)
        img = Image.open(img_path).convert("RGB")

        if self.transforms is not None:
            image = self.transforms(img)

        return image, label

In [55]:
transforms_valid = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
    ]
)

test_dataset = SDSSDataset(test, transforms=transforms_valid)
test_sampler = torch.utils.data.distributed.DistributedSampler(
        test_dataset,
        num_replicas=xm.xrt_world_size(),
        rank=xm.get_ordinal(),
        shuffle=False,
    )

test_loader = torch.utils.data.DataLoader(
dataset=test_dataset,
batch_size=1,
sampler=test_sampler,
drop_last=True,
num_workers=8,

)

In [59]:
!rm -r images
!mkdir images
!mkdir images/test

In [60]:
import shutil

for img in test['image']:
    shutil.copy("../input/sdss-images/images (1)/images/"+img,"./images/test/"+img)
    

In [61]:
for x in test_loader:
    print(len(x))
    break

2


In [62]:
model_inf = model_inference.eval()

In [63]:
model_inf(x[0])

tensor([[ 3.2374, -2.8192, -3.7498]], grad_fn=<AddmmBackward>)

In [64]:
device = xm.xla_device()

model_inf.to(device)
print("moved")

2022-10-24 16:12:24.537690: I torch_xla/csrc/tensor_util.cpp:28] Using BF16 data type for floating point values


moved


In [65]:
test_preds = list()

# device = xm.xla_device()
# model.to(device)

for x in tqdm(test_loader):
    test_preds.append(model_inf(x[0].to(device)))

  0%|          | 0/1000 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>Exception ignored in: Exception ignored in: Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>Exception ignored in: 
Exception ignored in: Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1079, in __del__
Traceback (most recent call last):
    
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1079, in __del__
Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>Traceback (most recent call last):
<function _MultiProcessingDataLoaderIter.__del__ at 0x7fe85673a050>    self._shutdown_worke

In [66]:
test_preds = [list(x.cpu().detach().numpy()[0]) for x in test_preds]

In [67]:
test_labels = [np.argmax(x) for x in test_preds]

In [68]:
# val_labels

In [69]:
true_labels = list()
for x in test_loader:
    true_labels.append(x[1].detach().cpu().numpy()[0])

In [70]:
from sklearn.metrics import classification_report

In [71]:
test_labels[:10],true_labels[:10]

([0, 2, 2, 2, 1, 2, 2, 0, 0, 2], [0, 2, 2, 2, 1, 2, 2, 0, 0, 2])

In [72]:
classification_report(test_labels,true_labels).split("\n")

['              precision    recall  f1-score   support',
 '',
 '           0       0.97      0.99      0.98       481',
 '           1       0.82      0.70      0.76       100',
 '           2       0.95      0.95      0.95       419',
 '',
 '    accuracy                           0.95      1000',
 '   macro avg       0.91      0.88      0.90      1000',
 'weighted avg       0.94      0.95      0.94      1000',
 '']